In [1]:
import numpy as np
from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField

from openff.interchange import Interchange
from openff.interchange.models import PotentialKey, TopologyKey
from openff.interchange.tests import get_test_file_path

In [2]:
# Load in a mainline OpenFF forcefield and construct a minimal ethanol topology
parsley = ForceField("openff-1.0.0.offxml")

top = Topology.from_molecules(2 * [Molecule.from_smiles("CCO")])

In [3]:
# Use a monkey-patched function to parametrize the topology against a force field
sys_out = Interchange.from_smirnoff(force_field=parsley, topology=top)

In [4]:
# Look at which ParameterHandler objects from the OpenFF toolkit
# have been made into Potentialhandler objects
sys_out.handlers.keys()

dict_keys(['Bonds', 'Constraints', 'Angles', 'ProperTorsions', 'ImproperTorsions', 'vdW', 'Electrostatics'])

In [5]:
# Store the bond handler to inspect its contents
bonds = sys_out.handlers["Bonds"]

In [6]:
# Look at some ~metadata
bonds.type, bonds.expression, bonds.independent_variables

('Bonds', 'k/2*(r-length)**2', {'r'})

In [7]:
# Return a mapping between atom indices and SMIRKS identifiers
bonds.slot_map

{TopologyKey(atom_indices=(0, 1), mult=None, bond_order=None): PotentialKey(id='[#6X4:1]-[#6X4:2]', mult=None, associated_handler='Bonds', bond_order=None),
 TopologyKey(atom_indices=(0, 3), mult=None, bond_order=None): PotentialKey(id='[#6X4:1]-[#1:2]', mult=None, associated_handler='Bonds', bond_order=None),
 TopologyKey(atom_indices=(0, 4), mult=None, bond_order=None): PotentialKey(id='[#6X4:1]-[#1:2]', mult=None, associated_handler='Bonds', bond_order=None),
 TopologyKey(atom_indices=(0, 5), mult=None, bond_order=None): PotentialKey(id='[#6X4:1]-[#1:2]', mult=None, associated_handler='Bonds', bond_order=None),
 TopologyKey(atom_indices=(1, 2), mult=None, bond_order=None): PotentialKey(id='[#6:1]-[#8:2]', mult=None, associated_handler='Bonds', bond_order=None),
 TopologyKey(atom_indices=(1, 6), mult=None, bond_order=None): PotentialKey(id='[#6X4:1]-[#1:2]', mult=None, associated_handler='Bonds', bond_order=None),
 TopologyKey(atom_indices=(1, 7), mult=None, bond_order=None): Potenti

In [8]:
# Return a mapping between SMIRKS identifiers and Potential objects;
# Note the de-duplication, resulting from a many-to-few mapping between
# bonds in the topology and unique parameters in the force field
bonds.potentials

{PotentialKey(id='[#6X4:1]-[#6X4:2]', mult=None, associated_handler='Bonds', bond_order=None): Potential(parameters={'k': <Quantity(531.137374, 'kilocalorie / angstrom ** 2 / mole')>, 'length': <Quantity(1.5203759, 'angstrom')>}, map_key=None),
 PotentialKey(id='[#6X4:1]-[#1:2]', mult=None, associated_handler='Bonds', bond_order=None): Potential(parameters={'k': <Quantity(758.093177, 'kilocalorie / angstrom ** 2 / mole')>, 'length': <Quantity(1.09288838, 'angstrom')>}, map_key=None),
 PotentialKey(id='[#6:1]-[#8:2]', mult=None, associated_handler='Bonds', bond_order=None): Potential(parameters={'k': <Quantity(669.141517, 'kilocalorie / angstrom ** 2 / mole')>, 'length': <Quantity(1.41428792, 'angstrom')>}, map_key=None),
 PotentialKey(id='[#8:1]-[#1:2]', mult=None, associated_handler='Bonds', bond_order=None): Potential(parameters={'k': <Quantity(1120.58324, 'kilocalorie / angstrom ** 2 / mole')>, 'length': <Quantity(0.970768759, 'angstrom')>}, map_key=None)}

In [9]:
# Look at this contents of one of the Potential objects
potential_key = PotentialKey(id="[#6X4:1]-[#1:2]", associated_handler="Bonds")
bonds.potentials[potential_key]

Potential(parameters={'k': <Quantity(758.093177, 'kilocalorie / angstrom ** 2 / mole')>, 'length': <Quantity(1.09288838, 'angstrom')>}, map_key=None)

In [10]:
# Further, look at the particular value of one of its parameters
bonds.potentials[potential_key].parameters["k"]

758.0931772913 <Unit('kilocalorie / angstrom ** 2 / mole')>

In [11]:
# Look up, from the highest-level object, this same data, using the
# SMIRKS pattern as a key connecting the topological data to the
# parametrized data
topology_key = TopologyKey(atom_indices=(1, 7))
sys_out.handlers["Bonds"].potentials[
    sys_out.handlers["Bonds"].slot_map[topology_key]
].parameters["k"]

758.0931772913 <Unit('kilocalorie / angstrom ** 2 / mole')>